In [1]:
from ib_insync import *
util.startLoop()

In [2]:
ib = IB()
ib.connect('', 7497, clientId=2)

<IB connected to :7497 clientId=2>

In [3]:
[v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']

[AccountValue(account='DU1113767', tag='NetLiquidationByCurrency', value='1017820.8326', currency='BASE', modelCode='')]

# Stocks

In [4]:
contract = Stock('MSFT', 'SMART', 'USD', primaryExchange='NASDAQ')

In [5]:
bars = ib.reqHistoricalData(contract,
                           endDateTime='',
                           durationStr='60 D',
                           barSizeSetting='1 hour',
                           whatToShow='Trades',
                           useRTH=True,
                           formatDate=1)

In [6]:
bars[-1]

BarData(date=datetime.datetime(2018, 8, 9, 9, 0), open=109.9, high=109.94, low=109.9, close=109.92, volume=575, barCount=374, average=109.914)

In [7]:
df = util.df(bars)
display(df.head())
display(df.tail())

/Users/lorinfields/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/lorinfields/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


,date,open,high,low,close,volume,barCount,average
0,2018-05-16 06:30:00,97.35,97.38,96.61,97.14,19549,9476,97.027
1,2018-05-16 07:00:00,97.15,97.34,96.87,97.01,25577,14121,97.171
2,2018-05-16 08:00:00,97.01,97.02,96.66,96.95,15988,9316,96.850
3,2018-05-16 09:00:00,96.96,97.40,96.95,97.25,15807,9686,97.199
4,2018-05-16 10:00:00,97.24,97.27,96.80,96.94,11847,7205,97.066


,date,open,high,low,close,volume,barCount,average
409,2018-08-08 12:00:00,109.43,109.75,109.40,109.48,29104,16925,109.573
410,2018-08-09 06:30:00,109.84,110.16,109.71,109.91,19989,9697,109.992
411,2018-08-09 07:00:00,109.92,110.05,109.71,109.94,18964,10801,109.932
412,2018-08-09 08:00:00,109.95,110.11,109.77,109.91,14078,8450,109.969
413,2018-08-09 09:00:00,109.90,109.94,109.90,109.92,575,374,109.914


In [40]:
def onBarUpdate(bars, hasNewBar):
    print(bars[-1])

ib.setCallback('barUpdate', onBarUpdate)

setCallback will be depreciated, use events instead


In [41]:
bars = ib.reqRealTimeBars(contract, 5, 'MIDPOINT', False)

RealTimeBar(time=datetime.datetime(2018, 8, 8, 19, 22, 5, tzinfo=datetime.timezone.utc), open=109.495, high=109.495, low=109.495, close=109.495, volume=-1, wap=-1.0, count=-1)


In [8]:
ib.sleep(10)
ib.cancelRealTimeBars(bars)

In [46]:
ticker = ib.reqMktDepth(contract)
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(5),
        columns='bidSize bidPrice askPrice askSize'.split())

for t in ib.loopUntil(timeout=15):
    l = ticker.domBids
for i in range(5):
    df.iloc[i, 0] = l[i].size if i < len(l) else 0
    df.iloc[i, 1] = l[i].price if i < len(l) else 0
l = ticker.domAsks
for i in range(5):
    df.iloc[i, 2] = l[i].price if i < len(l) else 0
    df.iloc[i, 3] = l[i].size if i < len(l) else 0
clear_output(wait=True)
display(df)

    
    

,bidSize,bidPrice,askPrice,askSize
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0


In [9]:
ib.cancelMktDepth(contract)

ERROR:ib_insync.ib:cancelMktDepth: No reqId found for contract Stock(symbol='MSFT', exchange='SMART', primaryExchange='NASDAQ', currency='USD')


In [11]:
ib.qualifyContracts(contract)

order = MarketOrder('BUY', 100)

In [12]:
trade = ib.placeOrder(contract, order)

In [13]:
ib.sleep(1)
trade.log


[TradeLogEntry(time=datetime.datetime(2018, 8, 9, 16, 3, 3, 495105, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''),
 TradeLogEntry(time=datetime.datetime(2018, 8, 9, 16, 3, 3, 729869, tzinfo=datetime.timezone.utc), status='PreSubmitted', message=''),
 TradeLogEntry(time=datetime.datetime(2018, 8, 9, 16, 3, 3, 833259, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='Fill 100.0@109.94'),
 TradeLogEntry(time=datetime.datetime(2018, 8, 9, 16, 3, 3, 833835, tzinfo=datetime.timezone.utc), status='Filled', message='')]

ERROR:ib_insync.wrapper:Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
ERROR:ib_insync.wrapper:Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained.
ERROR:ib_insync.wrapper:Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
ERROR:ib_insync.wrapper:Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained.
ERROR:ib_insync.client:Peer closed connection


# Futures


In [52]:
contract = Future('ES', '20180921', 'GLOBEX')

In [ ]:
bars = ib.reqHistoricalData(contract,
                           endDateTime='',
                           durationStr='60 D',
                           barSizeSetting='1 hour',
                           whatToShow='Trades',
                           useRTH=True,
                           formatDate=1)

In [54]:
bars[0]

BarData(date=datetime.datetime(2018, 5, 15, 6, 30), open=2721.0, high=2721.0, low=2707.25, close=2707.5, volume=335, barCount=128, average=2713.175)

In [55]:
df = util.df(bars)

display(df.head())
display(df.tail())

,date,open,high,low,close,volume,barCount,average
0,2018-05-15 06:30:00,2721.00,2721.0,2707.25,2707.50,335,128,2713.175
1,2018-05-15 07:00:00,2707.00,2719.5,2707.00,2713.25,349,173,2714.500
2,2018-05-15 08:00:00,2714.75,2718.0,2710.25,2711.00,110,60,2714.125
3,2018-05-15 09:00:00,2711.50,2718.0,2707.25,2714.50,204,98,2711.750
4,2018-05-15 10:00:00,2714.50,2714.5,2705.25,2705.25,84,52,2710.850


,date,open,high,low,close,volume,barCount,average
530,2018-08-08 09:00:00,2857.75,2859.5,2855.50,2857.75,52308,8350,2857.525
531,2018-08-08 10:00:00,2857.50,2860.5,2857.00,2860.00,40521,5721,2858.350
532,2018-08-08 11:00:00,2860.25,2861.0,2859.00,2859.75,40427,5507,2859.975
533,2018-08-08 12:00:00,2859.75,2862.5,2857.25,2857.75,119646,12759,2859.975
534,2018-08-08 13:00:00,2858.00,2858.5,2856.75,2857.00,25899,1881,2857.475


In [56]:
def onBarUpdate(bars, hasNewBar):
        print(bars[-1])
        
ib.setCallback('barUpdate', onBarUpdate)

setCallback will be depreciated, use events instead


In [57]:
bars = ib.reqRealTimeBars(contract, 5, 'MIDPOINT', False)

RealTimeBar(time=datetime.datetime(2018, 8, 8, 20, 9, tzinfo=datetime.timezone.utc), open=2856.125, high=2856.125, low=2856.125, close=2856.125, volume=-1, wap=-1.0, count=-1)
RealTimeBar(time=datetime.datetime(2018, 8, 8, 20, 9, 5, tzinfo=datetime.timezone.utc), open=2856.125, high=2856.125, low=2856.125, close=2856.125, volume=-1, wap=-1.0, count=-1)
RealTimeBar(time=datetime.datetime(2018, 8, 8, 20, 9, 10, tzinfo=datetime.timezone.utc), open=2856.125, high=2856.125, low=2856.125, close=2856.125, volume=-1, wap=-1.0, count=-1)
RealTimeBar(time=datetime.datetime(2018, 8, 8, 20, 9, 15, tzinfo=datetime.timezone.utc), open=2856.125, high=2856.375, low=2856.125, close=2856.375, volume=-1, wap=-1.0, count=-1)
RealTimeBar(time=datetime.datetime(2018, 8, 8, 20, 9, 20, tzinfo=datetime.timezone.utc), open=2856.375, high=2856.375, low=2856.375, close=2856.375, volume=-1, wap=-1.0, count=-1)
RealTimeBar(time=datetime.datetime(2018, 8, 8, 20, 9, 25, tzinfo=datetime.timezone.utc), open=2856.375, h

In [58]:
ib.sleep(10)
ib.cancelRealTimeBars(bars)

RealTimeBar(time=datetime.datetime(2018, 8, 8, 20, 10, 50, tzinfo=datetime.timezone.utc), open=2856.375, high=2856.375, low=2856.375, close=2856.375, volume=-1, wap=-1.0, count=-1)
RealTimeBar(time=datetime.datetime(2018, 8, 8, 20, 10, 55, tzinfo=datetime.timezone.utc), open=2856.375, high=2856.375, low=2856.375, close=2856.375, volume=-1, wap=-1.0, count=-1)


In [59]:
ticker = ib.reqMktDepth(contract)
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(5),
        columns='bidSize bidPrice askPrice askSize'.split())

for t in ib.loopUntil(timeout=15):
    l = ticker.domBids
for i in range(5):
    df.iloc[i, 0] = l[i].size if i < len(l) else 0
    df.iloc[i, 1] = l[i].price if i < len(l) else 0
l = ticker.domAsks
for i in range(5):
    df.iloc[i, 2] = l[i].price if i < len(l) else 0
    df.iloc[i, 3] = l[i].size if i < len(l) else 0
clear_output(wait=True)
display(df)

,bidSize,bidPrice,askPrice,askSize
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0


In [60]:
ib.cancelMktDepth(contract)

ERROR:ib_insync.wrapper:Error 310, reqId 11: Can't find the subscribed market depth with tickerId:11


In [63]:
ib.qualifyContracts(contract)

order = MarketOrder('BUY', 5)

In [64]:
trade = ib.placeOrder(contract, order)

# FOREX

In [65]:
contract = Forex('USDJPY')

In [66]:
bars = ib.reqHistoricalData(contract,
                           endDateTime='',
                           durationStr='60 D',
                           barSizeSetting='1 hour',
                           whatToShow='BID',
                           useRTH=False,
                           formatDate=1)

In [76]:
bars[-1]

RealTimeBar(time=datetime.datetime(2018, 8, 8, 20, 53, 25, tzinfo=datetime.timezone.utc), open=110.9485, high=110.949, low=110.9485, close=110.949, volume=-1, wap=-1.0, count=-1)

In [77]:
df = util.df(bars)

display(df.head())
display(df.tail())

,time,endTime,open,high,low,close,volume,wap,count
0,2018-08-08 20:52:45+00:00,-1,110.9450,110.9460,110.9445,110.9445,-1,-1.0,-1
1,2018-08-08 20:52:50+00:00,-1,110.9445,110.9455,110.9425,110.9445,-1,-1.0,-1
2,2018-08-08 20:52:55+00:00,-1,110.9445,110.9450,110.9445,110.9450,-1,-1.0,-1
3,2018-08-08 20:53:00+00:00,-1,110.9450,110.9450,110.9445,110.9445,-1,-1.0,-1
4,2018-08-08 20:53:05+00:00,-1,110.9445,110.9485,110.9445,110.9480,-1,-1.0,-1


,time,endTime,open,high,low,close,volume,wap,count
4,2018-08-08 20:53:05+00:00,-1,110.9445,110.9485,110.9445,110.9480,-1,-1.0,-1
5,2018-08-08 20:53:10+00:00,-1,110.9480,110.9480,110.9480,110.9480,-1,-1.0,-1
6,2018-08-08 20:53:15+00:00,-1,110.9480,110.9485,110.9480,110.9485,-1,-1.0,-1
7,2018-08-08 20:53:20+00:00,-1,110.9485,110.9485,110.9485,110.9485,-1,-1.0,-1
8,2018-08-08 20:53:25+00:00,-1,110.9485,110.9490,110.9485,110.9490,-1,-1.0,-1


In [78]:
def onBarUpdate(bars, hasNewBar):
        print(bars[-1])
        
ib.setCallback('barUpdate', onBarUpdate)

setCallback will be depreciated, use events instead


In [80]:
bars = ib.reqRealTimeBars(contract, 5, 'MIDPOINT', False)

In [81]:
ib.sleep(10)
ib.cancelRealTimeBars(bars)

In [82]:
ticker = ib.reqMktDepth(contract)
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(5),
        columns='bidSize bidPrice askPrice askSize'.split())

for t in ib.loopUntil(timeout=15):
    l = ticker.domBids
for i in range(5):
    df.iloc[i, 0] = l[i].size if i < len(l) else 0
    df.iloc[i, 1] = l[i].price if i < len(l) else 0
l = ticker.domAsks
for i in range(5):
    df.iloc[i, 2] = l[i].price if i < len(l) else 0
    df.iloc[i, 3] = l[i].size if i < len(l) else 0
clear_output(wait=True)
display(df)

,bidSize,bidPrice,askPrice,askSize
0,1000000,110.948,110.952,2000000
1,6000000,110.947,110.953,3500000
2,2000000,110.946,110.954,12000000
3,4000000,110.945,110.955,3000000
4,1000000,110.944,110.957,1000000


In [83]:
ib.cancelMktDepth(contract)

In [84]:
ib.qualifyContracts(contract)

order = MarketOrder('BUY', 10000)

In [86]:
trade = ib.placeOrder(contract, order)

# Options

In [109]:
contract = Stock('IBM', exchange='SMART', primaryExchange='NYSE')

ib.qualifyContracts(contract)

[Stock(conId=8314, symbol='IBM', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='IBM', tradingClass='IBM')]

In [110]:
chains = ib.reqSecDefOptParams(contract.symbol, '',
                                contract.secType, contract.conId)

util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,NASDAQOM,8314,IBM,100,"{20180824, 20180907, 20180817, 20180831, 20180...","{128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134..."
1,PSE,8314,IBM,100,"{20180824, 20180907, 20180817, 20180831, 20180...","{128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134..."
2,PHLX,8314,IBM,100,"{20180824, 20180907, 20180817, 20180831, 20180...","{128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134..."
3,BOX,8314,IBM,100,"{20180824, 20180907, 20180817, 20180831, 20180...","{128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134..."
4,PEARL,8314,IBM,100,"{20180824, 20180907, 20180817, 20180831, 20180...","{128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134..."
5,CBOE2,8314,IBM,100,"{20180824, 20180907, 20180817, 20180831, 20180...","{128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134..."
6,EDGX,8314,IBM,100,"{20180824, 20180907, 20180817, 20180831, 20180...","{128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134..."
7,GEMINI,8314,IBM,100,"{20180824, 20180907, 20180817, 20180831, 20180...","{128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134..."
8,MERCURY,8314,IBM,100,"{20180824, 20180907, 20180817, 20180831, 20180...","{128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134..."
9,MIAX,8314,IBM,100,"{20180824, 20180907, 20180817, 20180831, 20180...","{128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134..."


In [111]:
ticker = 'SPY'
contract = Stock(ticker, exchange='SMART', primaryExchange='NYSE')

ib.qualifyContracts(contract)

[Stock(conId=756733, symbol='SPY', exchange='SMART', primaryExchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY')]

In [112]:
chains = ib.reqSecDefOptParams(contract.symbol, '',
                                contract.secType, contract.conId)

util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,NASDAQOM,756733,SPY,100,"{20180824, 20200619, 20180831, 20180815, 20200...","{346.0, 306.0, 306.5, 307.0, 10.0, 307.5, 308...."
1,PSE,756733,SPY,100,"{20180824, 20200619, 20180831, 20180815, 20200...","{346.0, 306.0, 306.5, 307.0, 10.0, 307.5, 308...."
2,BATS,756733,SPY,100,"{20180824, 20200619, 20180831, 20180815, 20200...","{346.0, 306.0, 306.5, 307.0, 10.0, 307.5, 308...."
3,AMEX,756733,SPY,100,"{20180824, 20200619, 20180831, 20180815, 20200...","{346.0, 306.0, 306.5, 307.0, 10.0, 307.5, 308...."
4,PEARL,756733,SPY,100,"{20180824, 20200619, 20180831, 20180815, 20200...","{346.0, 306.0, 306.5, 307.0, 10.0, 307.5, 308...."
5,NASDAQBX,756733,SPY,100,"{20180824, 20200619, 20180831, 20180815, 20200...","{346.0, 306.0, 306.5, 307.0, 10.0, 307.5, 308...."
6,ISE,756733,SPY,100,"{20180824, 20200619, 20180831, 20180815, 20200...","{346.0, 306.0, 306.5, 307.0, 10.0, 307.5, 308...."
7,MIAX,756733,SPY,100,"{20180824, 20200619, 20180831, 20180815, 20200...","{346.0, 306.0, 306.5, 307.0, 10.0, 307.5, 308...."
8,SMART,756733,SPY,100,"{20180824, 20200619, 20180831, 20180815, 20200...","{346.0, 306.0, 306.5, 307.0, 10.0, 307.5, 308...."
9,CBOE,756733,SPY,100,"{20180824, 20200619, 20180831, 20180815, 20200...","{346.0, 306.0, 306.5, 307.0, 10.0, 307.5, 308...."


In [113]:
chain = next(c for c in chains if c.exchange =='SMART')
chain

OptionChain(exchange='SMART', underlyingConId=756733, tradingClass='SPY', multiplier='100', expirations={'20180824', '20200619', '20180831', '20180815', '20200117', '20180912', '20190329', '20190118', '20190621', '20200320', '20180907', '20180914', '20180829', '20180928', '20180810', '20180910', '20180808', '20191220', '20190628', '20190920', '20181221', '20190315', '20180820', '20201218', '20180905', '20180813', '20180817', '20180921', '20180827', '20181116', '20181231', '20180822', '20181019', '20180904'}, strikes={346.0, 306.0, 306.5, 307.0, 10.0, 307.5, 308.5, 15.0, 20.0, 245.5, 25.0, 311.0, 30.0, 312.0, 35.0, 312.5, 254.5, 40.0, 45.0, 246.5, 50.0, 316.0, 55.0, 317.5, 60.0, 65.0, 70.0, 247.5, 75.0, 80.0, 321.0, 85.0, 322.5, 253.5, 90.0, 95.0, 248.5, 100.0, 105.0, 326.0, 110.0, 327.5, 115.0, 120.0, 249.5, 125.0, 130.0, 331.0, 135.0, 332.0, 140.0, 145.0, 250.5, 150.0, 155.0, 336.0, 160.0, 165.0, 170.0, 251.5, 174.5, 175.0, 175.5, 176.0, 176.5, 177.0, 178.0, 179.0, 180.0, 181.0, 182.0

In [114]:
[ticker_details] = ib.reqTickers(contract)

In [115]:
import datetime

def isFriday(date):
    y = int(date[0:4])
    m = int(date[4:6])    
    d = int(date[6:8])    
    dd = datetime.date(y,m,d)
    return dd.weekday() == 4 and dd > datetime.date.today()


In [116]:
strikes = [strike for strike in chain.strikes if 
           strike % 5 == 0 and 
            ticker_details.marketPrice() - 20 < strike < ticker_details.marketPrice() + 20]
expirations = sorted(exp for exp in chain.expirations if isFriday(exp))[:3]
rights = ['P','C']

contracts = [Option(ticker, expiration, strike, right, 'SMART')
             for right in rights for expiration in expirations for strike in strikes]

ib.qualifyContracts(*contracts)

print(len(contracts))
print()
print(contracts[0])

48

Option(conId=324254475, symbol='SPY', lastTradeDateOrContractMonth='20180810', strike=265.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   180810P00265000', tradingClass='SPY')


In [102]:
tickers_data = ib.reqTickers(*contracts)

tickers_data[-1]

RealTimeBar(time=datetime.datetime(2018, 8, 8, 21, 43, 5, tzinfo=datetime.timezone.utc), open=110.925, high=110.925, low=110.924, close=110.924, volume=-1, wap=-1.0, count=-1)
RealTimeBar(time=datetime.datetime(2018, 8, 8, 21, 43, 10, tzinfo=datetime.timezone.utc), open=110.924, high=110.924, low=110.924, close=110.924, volume=-1, wap=-1.0, count=-1)


Ticker(contract=Option(conId=324945729, symbol='SPY', lastTradeDateOrContractMonth='20180824', strike=305.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   180824C00305000', tradingClass='SPY'), time=datetime.datetime(2018, 8, 8, 21, 43, 8, 87828, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, close=0.01, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])

RealTimeBar(time=datetime.datetime(2018, 8, 8, 21, 43, 15, tzinfo=datetime.timezone.utc), open=110.924, high=110.924, low=110.923, close=110.923, volume=-1, wap=-1.0, count=-1)
RealTimeBar(time=datetime.datetime(2018, 8, 8, 21, 43, 20, tzinfo=datetime.timezone.utc), open=110.923, high=110.927, low=110.9225, close=110.926, volume=-1, wap=-1.0, count=-1)
RealTimeBar(time=datetime.datetime(2018, 8, 8, 21, 43, 25, tzinfo=datetime.timezone.utc), open=110.926, high=110.926, low=110.926, close=110.926, volume=-1, wap=-1.0, count=-1)
RealTimeBar(time=datetime.datetime(2018, 8, 8, 21, 43, 30, tzinfo=datetime.timezone.utc), open=110.926, high=110.926, low=110.9215, close=110.9225, volume=-1, wap=-1.0, count=-1)
RealTimeBar(time=datetime.datetime(2018, 8, 8, 21, 43, 35, tzinfo=datetime.timezone.utc), open=110.9225, high=110.924, low=110.9215, close=110.9235, volume=-1, wap=-1.0, count=-1)
RealTimeBar(time=datetime.datetime(2018, 8, 8, 21, 43, 40, tzinfo=datetime.timezone.utc), open=110.9235, high

In [119]:
tickers_data[8]

Ticker(contract=Option(conId=315359823, symbol='SPY', lastTradeDateOrContractMonth='20180817', strike=270.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   180817P00270000', tradingClass='SPY'), time=datetime.datetime(2018, 8, 8, 21, 43, 8, 87200, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, last=0.07, lastSize=60, volume=522, high=0.09, low=0.06, close=0.08, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])

In [120]:
tickers_data[8].contract

Option(conId=315359823, symbol='SPY', lastTradeDateOrContractMonth='20180817', strike=270.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   180817P00270000', tradingClass='SPY')

In [122]:
ib.qualifyContracts(tickers_data[8].contract)

order = MarketOrder('BUY', 5)

### placeOrder will place the order and return a Trade object right away (non-blocking)

In [123]:
trade = ib.placeOrder(contract, order)

trade

Trade(contract=Stock(conId=756733, symbol='SPY', exchange='SMART', primaryExchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY'), order=MarketOrder(orderId=58, action='BUY', totalQuantity=5), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2018, 8, 8, 22, 21, 43, 536863, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

### trade contains the order and everything related to it. such as order status fills and a log. it live pdates with status changes or fill of order

In [124]:
ib.sleep(1)
trade.log

[TradeLogEntry(time=datetime.datetime(2018, 8, 8, 22, 21, 43, 536863, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''),
 TradeLogEntry(time=datetime.datetime(2018, 8, 8, 22, 21, 43, 731857, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='')]

In [127]:
ib.disconnect()

ERROR:ib_insync.client:Peer closed connection
ERROR:ib_insync.client:Peer closed connection
